In [1]:
from datasets import load_dataset, concatenate_datasets
import json
import torch
from datasets.utils.logging import disable_progress_bar,enable_progress_bar

dataset_name = "super_glue"  # The MMLU dataset name in Hugging Face Datasets
dataset = load_dataset(dataset_name, name='rte')#["train"]
# print(dataset)

labels = dataset["train"].features["label"]
# def create_label_str(batch):
#     return {"label_str": labels.int2str(batch["label"])}
def create_label_str(example):
    return {"label_str": "False" if example["label"]== 1 else "True"}
# print(labels)


/home/hshapour/.pyenv/versions/3.10.12/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 3000/3000 [00:00<00:00, 22185.08 examples/s]


In [16]:
from datasets import Dataset
my_list = [{"a": 1}, {"a": 2}, {"a": 3}]
dataset = Dataset.from_list(my_list)
print(dataset)

Dataset({
    features: ['a'],
    num_rows: 3
})


In [2]:

# data_train = dataset["train"].map(create_label_str)   
# data_test = dataset["validation"].map(create_label_str)   
# # print(d1)
# # dd = d1.select(range(5)).shuffle()

data_train = dataset["train"].map(create_label_str)   
data_test = dataset["validation"].map(create_label_str)   

# # d_all = concatenate_datasets([d1,d2])
# data_train = d_all.select(range(5))
# data_test = d_all.select(range(5,d_all.num_rows))

print(data_train.num_rows, data_test.num_rows)

Map: 100%|██████████| 277/277 [00:00<00:00, 12406.40 examples/s]

2490 277


In [3]:


def json_writer(JSON_PATH, num_qs, n_shot_example,seed=10,space=False):
    qlist = {}
    # if n_shot_example >0:
    #     instruction = 'The following are multiple choice questions (with answers) about  '
    # else:
    #     instruction = 'The following is a multiple choice question about  '
    with open(JSON_PATH, 'w') as json_file:

        for ind in range(num_qs):
            dd_examples = data_train.select(range(5)).shuffle()
            prompt = ''
            for idx in range(n_shot_example):
                prompt += "Text: " + dd_examples[idx]['premise'] + '\n'
                prompt += "Hypothesis: " + dd_examples[idx]['hypothesis'] + '\n'
                prompt += "Entailment:" + dd_examples[idx]['label_str'] + '\n\n'

            prompt += "Text: " + data_test[ind]['premise'] + '\n'
            prompt += "Hypothesis: " + data_test[ind]['hypothesis'] + '\n'
            prompt += "Entailment:" 
    
            
            qlist['input'] = prompt
            qlist['target_str'] = data_test[ind]['label_str']
            qlist['target'] = (2**data_test[ind]['label']) % 2

            # Serializing json
            json_file.write(json.dumps(qlist) + '\n')
        # print("Done!")

In [4]:
disable_progress_bar()

# print(example_idx)
n_example = data_test.num_rows
n_shot_vals = [2]
seed = 123
for n_shot in range(6):
# for n_shot in n_shot_vals:
    print(f"{n_shot}-shot example is being generated...")
    fname_json = f"rte-data/{n_shot}_shot_examples.json"
    # fname_json = f"boolq-data/{n_shot}_shot_examples_small.json"
    # JSON_eval = 'sample_eval.json'
    json_writer(fname_json, n_example, n_shot, seed=seed, space= False)
    # json_writer(JSON_eval,example_idx[(n_shot_example+1)*n_test_ex:])

enable_progress_bar()


0-shot example is being generated...
1-shot example is being generated...
2-shot example is being generated...
3-shot example is being generated...
4-shot example is being generated...
5-shot example is being generated...


In [4]:
# fname_json = f"mmlu-data/ex_no_space.json"
n_shot = 0
d1 = load_dataset("json", data_files={
        'dev' : f"boolq-data/{n_shot}_shot_examples.json"
    })
# print(d1['dev'])
# print(d1['dev'].filter(lambda example: example["subject"] == subject))
# d2 = d1['dev'].filter(lambda example: example["subject"] == subject)
# # print(d2['input'][0][-1])
# # print("---------")
print(d1['dev'][2]['input'])

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1433.95it/s]
Generating dev split: 3270 examples [00:00, 158649.11 examples/s]

Passage: Phantom pain -- Phantom pain sensations are described as perceptions that an individual experiences relating to a limb or an organ that is not physically part of the body. Limb loss is a result of either removal by amputation or congenital limb deficiency. However, phantom limb sensations can also occur following nerve avulsion or spinal cord injury.
Question: is pain experienced in a missing body part or paralyzed area
Answer:


In [5]:
from datasets import DatasetDict, Dataset
from collections import defaultdict

fname_dict = {}
for n_shot in range(6):
    fname_dict[f"{n_shot}_shot"] = f"rte-data/{n_shot}_shot_examples.json"
# data_all = defaultdict(DatasetDict)
d_upload = load_dataset("json", data_files=fname_dict)
    # data_all[f"n_shot = {n_shot}"] = d1['dev']


Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 1491.31it/s]
Generating 0_shot split: 277 examples [00:00, 39765.28 examples/s]
Generating 1_shot split: 277 examples [00:00, 53306.82 examples/s]
Generating 2_shot split: 277 examples [00:00, 76465.86 examples/s]
Generating 3_shot split: 277 examples [00:00, 46469.17 examples/s]
Generating 4_shot split: 277 examples [00:00, 46118.70 examples/s]
Generating 5_shot split: 277 examples [00:00, 35149.22 examples/s]


In [6]:
# print(Dataset.from_dict(data_all))
# Dataset.from_dict(data_all).push_to_hub("hassansh/boolq_n_shot")
d_upload.push_to_hub("hassansh/rte_n_shot")


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
README.md: 100%|██████████| 1.33k/1.33k [00:00<00:00, 1.55MB/s]


In [16]:
print(d_upload['5_shot'][2]['input'])

Text: Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.
Hypothesis: Herceptin can be used to treat breast cancer.
Entailment:True

Text: Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.
Hypothesis: The previous name of Ho Chi Minh City was Saigon.
Entailment:True

Text: A man is due in court later charged with the murder 26 years ago of a teenager whose case was the first to be featured on BBC One's Crimewatch. Colette Aram, 16, was walking to her boyfriend's house in Keyworth, Nottinghamshire, on 30 October 1983 when she disappeared. Her body was later found in a field close to her home. Paul Stewart Hutchinson, 50, has been charged with mur

In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3245
    })
})
